- 保监会 分类模型 2 训练 

# 基本设置

In [27]:
##load packages, needed
# encoding=utf-8

import jieba
from jieba import analyse
import sys
import re
import time
import string
from sklearn import feature_extraction
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2,mutual_info_classif,f_classif
from sklearn.preprocessing import StandardScaler

import xgboost as xgb
from xgboost import XGBClassifier
from sklearn import metrics

from sklearn.base import BaseEstimator, TransformerMixin
from collections import defaultdict

import pickle

%matplotlib inline
import numpy as np
import pandas as pd
import pre_cor
import dict_dbutils
import os
from sklearn.model_selection import train_test_split
from sqlalchemy import create_engine
from pandas.io import sql
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

In [77]:
class StatsFeatures_cor(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        self.neg = set()
        f = open("corpus/neg_words.txt","r+", encoding='UTF-8')
        for content in f:
            self.neg.add(content.strip())
        f.close()
        
        self.company = set() # 公司
        f = open("corpus/insurance_company_20180803.txt","r+", encoding='UTF-8')
        for content in f:
            self.company.add(content.strip())
        f.close()

        self.regulators = set() # 监管机构及领导
        f = open("corpus/insurance_regulators_20180804.txt","r+", encoding='UTF-8')
        for content in f:
            self.regulators.add(content.strip())
        f.close()        
        
    def fit(self, X, y=None):
        return self

    def getcnt(self,x):        
        return len(list(set(x)))

    def getnegcnt(self,x):
        negcnt = 0
        words = x.split()
        for w in words:
            if w in self.neg:
                negcnt = negcnt+1
        return negcnt
    
    def getorgcnttf(self,x):
        companycnt=0
        companytf=0
        regcnt = 0
        regtf = 0
        
        words = x.split()
        words_set=set(words)
        for w in words_set:
            if w in self.company:
                companycnt = companycnt+1
                companytf=companytf+words.count(w)
                
            if w in self.regulators:
                regcnt = regcnt+1
                regtf=regtf+words.count(w)            
                
        return companycnt, companytf, regcnt, regtf
    
    def transform(self, X):
        data = []
        for x in X:
            if len(x) == 0:
                length  = 1
            else :
                length = len(x)
            companycnt, companytf, regcnt, regtf=self.getorgcnttf(x)
            data.append([len(x),self.getcnt(x),self.getcnt(x)/length,
                         self.getnegcnt(x),self.getnegcnt(x)/length,
                         companycnt, companytf, regcnt, regtf])            
        return data

In [43]:
ss = StatsFeatures_cor()

In [62]:
X_test[a[0]]

'保险业 监管 持续 新年 首周 开出 千万 罚单 保险业 监管 持续 新年 首周 开出 千万 罚单 图片 新华网 确定 不再 此人 确定 取消 毫无疑问 保险行业 监管 号文 以及 相关 规章制度 大量 下发 各种 检查 罚单 监管函 数量 激增 据统计 年仅 保监会 官网 公布 相关 政策 通知 多份 人身险 财产险 相关 文件 超过 种种 迹象 显示 保险业 监管 持续 进入 第一周 保险行业 开出 罚单 罚款 千万元 保监会 发出 一份 风险 提示 直指 保险公司 开年 业绩 举足轻重 保险产品 开门红 销售 保险业 监管 持续 新年 首周 开出 罚单 处罚 保险公司 既有 公司 中小 保险公司 处罚 原因 多种多样 罚单 罚款 金额 合计 万元 例如 北京保监局 发出 两份 处罚 决定书 一份 复星保德信 北京 分公司 行政处罚 原因 公司 存在 按照 规定 报送 保管 产品 说明会 录音 录像 资料 逾期 改正 行为 被罚 万元 相关 负责人 共计 被罚 万元 一份 中国人寿 北京 分公司 行政处罚 原因 中国人寿 北京 分公司 存在 欺骗 投保人 行为 被罚 相关 负责人 被警告 合计 被罚 万元 民生人寿 福建 分公司 更是 五份 罚单 共计 被罚 万元 公司 处罚 原因 包括 欺骗 投保人 承诺 给予 投保人 保险合同 约定 以外 其他 利益 保险产品 结算 利率 银行存款 利率 国债 利率 银行 理财产品 收益率 进行 片面 对比 以及 妨碍 依法 监督 检查 问题 此外 海航 机票 违规 搭售保险 被罚 海航 销售 公司 海南 监管局 提出 陈述 申辩 驳回 开门红 遇强 监管 每逢 岁末年初 保险公司 开展 开门红 活动 开门红 现有 销售 行业 特有 一个 销售 节点 促进 产品 销售 在此期间 产品 宣传 推广 以及 销售 推动 力度 相对 较大 开门红 对于 保险公司 十分 重要 往年 情况 开门红 业绩 甚至 全年 保费 三分之一 甚至 二分之一 比重 业内 一直 开门红 全年 说法 开门红 新年 遭遇 监管 保监会 发布 专门 针对 开门红 风险 提示 直指 以往 开门红 期间 存在 乱象 保监会 开门红 期间 业绩 保险销售人员 利用 活动 炒作 概念 即将 停售 限时 限量 产品 打折 概念 消费者 推销产品 诱导 冲动 购买 

In [63]:
X_test[b[0]]

'第六届 中国 保险 产业 国际 峰会 上海 隆重召开 导读 保险 春风 已至 机遇 挑战 并存 第六届 中国 保险 产业 国际 峰会 上海 隆重召开 投保 图片 中国 上海 我国 保险业 注定 平凡 一年 关于 规范 金融机构 资产 管理 业务 指导 意见 资管 新规 落地 银保合并 金融监管 重大 改革 中国人民银行 行长 易纲 博鳌 亚洲 论坛 宣布 加速 保险业 对外开放 时间表 我国 保险 深度 密度 比起 发达国家 处于 低水平 潜力 巨大 如今 机遇 挑战 并存 我国 保险业 迎来 新一轮 发展 面对 外资保险 机构 带来 竞争 中国 保险 企业 顺势 提升 自我 专业 水平 进行 产品 服务 创新 运用 先进 营销 理念 经营 模式 拥抱 互联网 数据 区块 链等科 服务 提供 客户 中心 差异化 服务 全面 提升 前后 运营 效率 推进 我国 保险业 健康 发展 图片 往届 会议 现场 回顾 上海 隆重召开 第六届 中国 保险 产业 国际 峰会 由士研 咨询 主办 国际 保险 学会 支持 环境 政策 新思维 技术 主题 邀请 来自 全球 监管 机构 国际 保险 学术 机构 国内外 主流 保险公司 保险经纪 公司 机构 投资者 资产 管理 公司 保险行业 研究 机构 咨询 公司 前沿 信息 科技 公司 软件技术 公司 第三方 网络平台 以及 律师 事务所 企业 高管 专家 共同 探讨 国际 保险 热点 趋势 深入 挖掘 对接 合作 机会 以往 不同 本届 为期 大会 一天 全体 大会 三场 平行 论坛 组成 包含 主题 演讲 圆桌 讨论 颁奖典礼 呈现 国际 保险行业 领袖 思想 碰撞 饕餮 盛宴 提供 集中 高效 发掘 潜在 合作项目 绝佳 机会 核心 议题 内容 大会 第一天 议题 环境 保险业 发展趋势 格局 议题 新形势下 保险 创新 发展 大会 第二天 论坛 保险 经营 创新 论坛 论坛 保险 科技 论坛 论坛 保险资管 论坛 正式 大会 第一天 上午 围绕 环境 保险 发展趋势 格局 展开 话题 探讨 中国人民大学 保险 研究所 所长 魏丽 发表 首场 言论 众所周知 数字化 时代 驱动 保险业 跟随 时间 不断 创新 发展 保险 如何 转型 数字 当下 热门话题 背景 法国安盛 首席 数字 LEE 带来 数字化 时代 保险业 创新 发展

In [60]:
ssss = ss.transform([X_test[a[0]]])
ssss

[[1713, 377, 0.22008172796263864, 31, 0.018096906012842966, 4, 5, 6, 23]]

In [64]:
ssss = ss.transform([X_test[b[0]]])
ssss

[[2481, 459, 0.18500604594921402, 0, 0.0, 11, 11, 4, 40]]

In [55]:
np.array(X_test)[a]

array(['保险业 监管 持续 新年 首周 开出 千万 罚单 保险业 监管 持续 新年 首周 开出 千万 罚单 图片 新华网 确定 不再 此人 确定 取消 毫无疑问 保险行业 监管 号文 以及 相关 规章制度 大量 下发 各种 检查 罚单 监管函 数量 激增 据统计 年仅 保监会 官网 公布 相关 政策 通知 多份 人身险 财产险 相关 文件 超过 种种 迹象 显示 保险业 监管 持续 进入 第一周 保险行业 开出 罚单 罚款 千万元 保监会 发出 一份 风险 提示 直指 保险公司 开年 业绩 举足轻重 保险产品 开门红 销售 保险业 监管 持续 新年 首周 开出 罚单 处罚 保险公司 既有 公司 中小 保险公司 处罚 原因 多种多样 罚单 罚款 金额 合计 万元 例如 北京保监局 发出 两份 处罚 决定书 一份 复星保德信 北京 分公司 行政处罚 原因 公司 存在 按照 规定 报送 保管 产品 说明会 录音 录像 资料 逾期 改正 行为 被罚 万元 相关 负责人 共计 被罚 万元 一份 中国人寿 北京 分公司 行政处罚 原因 中国人寿 北京 分公司 存在 欺骗 投保人 行为 被罚 相关 负责人 被警告 合计 被罚 万元 民生人寿 福建 分公司 更是 五份 罚单 共计 被罚 万元 公司 处罚 原因 包括 欺骗 投保人 承诺 给予 投保人 保险合同 约定 以外 其他 利益 保险产品 结算 利率 银行存款 利率 国债 利率 银行 理财产品 收益率 进行 片面 对比 以及 妨碍 依法 监督 检查 问题 此外 海航 机票 违规 搭售保险 被罚 海航 销售 公司 海南 监管局 提出 陈述 申辩 驳回 开门红 遇强 监管 每逢 岁末年初 保险公司 开展 开门红 活动 开门红 现有 销售 行业 特有 一个 销售 节点 促进 产品 销售 在此期间 产品 宣传 推广 以及 销售 推动 力度 相对 较大 开门红 对于 保险公司 十分 重要 往年 情况 开门红 业绩 甚至 全年 保费 三分之一 甚至 二分之一 比重 业内 一直 开门红 全年 说法 开门红 新年 遭遇 监管 保监会 发布 专门 针对 开门红 风险 提示 直指 以往 开门红 期间 存在 乱象 保监会 开门红 期间 业绩 保险销售人员 利用 活动 炒作 概念 即将 停售 限时 限量 产品 打折 概念 消费者 推销产品 诱导

In [61]:
a = [i for i,v in enumerate(y_test) if v == 1]
len(a)
b = [i for i,v in enumerate(y_test) if v == 2]
len(b)

97

In [29]:
def getkeywords(X, N = 1000):
    '''
    训练时生成，合并所有记录，取N个关键词
    '''
    textrank = analyse.textrank

    text_combined = ' '.join(X)
    keywords = textrank(text_combined, topK = N)
    print('keywords num: ', len(keywords))
    if len(keywords) < N : 
        N  = len(keywords)

    if keywords:
        f = open("corpus/keywords.txt","w+", encoding='UTF-8')
        for content in keywords:
            content = content.strip()
            f.write(content + '\n')
        f.close()

In [30]:
class Statskeywords(BaseEstimator, TransformerMixin):
    
    def __init__(self, topk = 100):
        self.topk = topk
#         print(self.topk)
        self.keywords = set()
        f = open("corpus/keywords.txt","r+", encoding='UTF-8')
        num = 0
        for content in f:
            if num < topk:
                self.keywords.add(content.strip().replace('\n', ''))
            num += 1
        f.close() 
        
        #初始化字典liwc
        self.liwc = {} 
        f2 = open("corpus/scliwc.txt",'r', encoding = 'gb18030')
        for ii in f2:     #ii在scliwc.txt中循环
            i = ii.strip().split() 
            self.liwc[i[0]] = i[1:len(i)]
        f2.close      
        
        self.category = set()
        for i in list(self.liwc.values()):
            for j in i:
                self.category.add(j)        
    
    def fit(self, X, y=None):
        return self 
    
    def transform(self, X):
        '''
        文本中关键词的词频
        '''                        
        data = []
        for x in X:
            words = x.split()
            word_tf = []
            keycnt = 0
            for kw in self.keywords:
                word_tf.append(words.count(kw)) # 各个关键词的词频
                if kw in words:keycnt+=1
            word_tf.append(keycnt) # 关键词的个数
            
            psy = []
            for w in words:
                if w in self.liwc: #是否liwc字典包含分词结果列表words的哪些分词
                    psy += self.liwc[w]  
            cat_tf = []
            for cat in self.category:
                cat_tf.append(psy.count(cat))                
                
            data.append(word_tf + cat_tf)            
        return data        

## 上一版模型读入

In [31]:
# 上一版模型
from sklearn.externals import joblib
pipeline_old = joblib.load( "model/circ_8classifier_0730_1.pkl.z") 

# 导入数据

In [65]:
title = []
filename = 'data/titles.txt'
fid = open(filename, "r+", encoding='UTF-8')
for f in fid:
    title.append(f.strip().replace('\n', ''))
fid.close()
print(len(title))

10530


In [66]:
title[:2]

['做好 规划 不当 月光族 守财奴', '车险 代理 前景 如何 信合 财富 事业 迈上 高度']

In [67]:
content = []
filename = 'data/contents.txt'
fid = open(filename, "r+", encoding='UTF-8')
for f in fid:
    content.append(f.strip().replace('\n', ''))
fid.close()
print(len(content))
# content[:2]

10530


In [68]:
title_content = [t + ' ' + c for t,c in zip(title, content)]
print(len(title_content))

10530


In [69]:
label_dic={'监管':1,'行业':2,'产品销售':3,'资本市场':4,'公司内部管理':5,'消费服务':6,'其他相关报道':7,'噪音':8}
class_name_dict = {v: k for k, v in label_dic.items()}
class_name_dict

{1: '监管',
 2: '行业',
 3: '产品销售',
 4: '资本市场',
 5: '公司内部管理',
 6: '消费服务',
 7: '其他相关报道',
 8: '噪音'}

In [70]:
label = []
filename = 'data/labels.txt'
fid = open(filename, "r+", encoding='UTF-8')
for f in fid:
    label.append(f.strip().replace('\n', ''))
fid.close()
print(len(label))
label[:5]

10530


['噪音', '噪音', '产品销售', '噪音', '消费服务']

In [71]:
label = [label_dic[l] for l in label]
label[:5]

[8, 8, 3, 8, 6]

# 训练模型

## 分割训练集、测试集

In [72]:
X_train, X_test, y_train, y_test = train_test_split(title_content, label, test_size=0.3, random_state=42)
print('训练集：',len(y_train))
print('训练集-各类数量：',Counter(y_train))
print('测试集：',len(y_test))
print('测试集-各类数量：',Counter(y_test))

训练集： 7371
训练集-各类数量： Counter({8: 2358, 6: 1117, 1: 1056, 7: 1005, 5: 638, 2: 520, 4: 410, 3: 267})
测试集： 3159
测试集-各类数量： Counter({8: 1018, 6: 456, 7: 451, 1: 432, 5: 263, 2: 243, 4: 173, 3: 123})


## xgboost

### train

In [73]:
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('tf_idf', Pipeline([
            ('counts', CountVectorizer(max_df=0.95, min_df=2)),
            ('tf_idf', TfidfTransformer()),
            ('chi', SelectKBest(chi2, k=20000))
        ])),
        ('tf', Statskeywords(topk = 1000)),
        ('len_stats', StatsFeatures_cor())
    ])),
    ('standard', StandardScaler(with_mean=False)),
    ('classifier', XGBClassifier(max_depth=7,objective='multi:softmax', num_class=8))
])

pipeline.fit(X_train, y_train)
print(pipeline.score(X_train, y_train))
pipeline

0.9974223307556641


Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('tf_idf', Pipeline(memory=None,
     steps=[('counts', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.9...tate=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1))])

### test

In [74]:
y_pred_class = pipeline.predict(X_test)
print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
print(metrics.classification_report(y_test, y_pred_class))
print('confusion_matrix: ')
print( metrics.confusion_matrix(y_test, y_pred_class))

accuracy_score:  0.8531180753402976
             precision    recall  f1-score   support

          1       0.80      0.88      0.84       432
          2       0.70      0.60      0.65       243
          3       0.77      0.51      0.61       123
          4       0.81      0.76      0.79       173
          5       0.82      0.82      0.82       263
          6       0.92      0.93      0.92       456
          7       0.88      0.90      0.89       451
          8       0.89      0.92      0.90      1018

avg / total       0.85      0.85      0.85      3159

confusion_matrix: 
[[378  14   3   4  10   0   3  20]
 [ 31 146   6   6   8  11  12  23]
 [  3   4  63   0   8  13  14  18]
 [  7  15   0 132   5   0   0  14]
 [  9   9   3   4 216   5   5  12]
 [  0   4   2   0   2 422  13  13]
 [  6   9   4   1   7   4 404  16]
 [ 37   7   1  16   9   6   8 934]]


# 其他模型-单一模型

## DecisionTreeClassifier

In [21]:
from sklearn.tree import DecisionTreeClassifier

In [62]:
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('tf_idf', Pipeline([
            ('counts', CountVectorizer(max_df=0.95, min_df=2)),
            ('tf_idf', TfidfTransformer()),
            ('chi', SelectKBest(chi2, k=20000))
        ])),
        ('tf', Statskeywords(topk = 1000)),
        ('len_stats', StatsFeatures_cor())
    ])),
    ('standard', StandardScaler(with_mean=False)),
    ('classifier', DecisionTreeClassifier(random_state=0))
])

pipeline.fit(X_train, y_train)
print(pipeline.score(X_train, y_train))
pipeline

0.9995004163197335


Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('tf_idf', Pipeline(memory=None,
     steps=[('counts', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.9...         min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best'))])

In [63]:
y_pred_class = pipeline.predict(X_test)
print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
print(metrics.classification_report(y_test, y_pred_class))
print('confusion_matrix: ')
print( metrics.confusion_matrix(y_test, y_pred_class))

accuracy_score:  0.7450066577896138
             precision    recall  f1-score   support

          1       0.75      0.76      0.76       279
          2       0.38      0.29      0.33        65
          3       0.37      0.40      0.38        63
          4       0.63      0.64      0.63        89
          5       0.59      0.69      0.64       128
          6       0.61      0.59      0.60        59
          7       0.75      0.69      0.72       143
          8       0.86      0.86      0.86       676

avg / total       0.74      0.75      0.74      1502

confusion_matrix: 
[[212   8   5   6  13   0   2  33]
 [ 13  19   1   4  14   2   3   9]
 [  6   4  25   1   4   6   8   9]
 [  9   2   1  57   4   2   1  13]
 [  7   4   8   5  88   1   6   9]
 [  1   0   8   2   6  35   4   3]
 [  4   3   3   3   9   6  99  16]
 [ 29  10  16  13  10   5   9 584]]


## SVC

In [24]:
from sklearn.svm import SVC

In [64]:
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('tf_idf', Pipeline([
            ('counts', CountVectorizer(max_df=0.95, min_df=2)),
            ('tf_idf', TfidfTransformer()),
            ('chi', SelectKBest(chi2, k=20000))
        ])),
        ('tf', Statskeywords(topk = 1000)),
        ('len_stats', StatsFeatures_cor())
    ])),
    ('standard', StandardScaler(with_mean=False)),
    ('classifier', SVC(random_state=0))
])

pipeline.fit(X_train, y_train)
print(pipeline.score(X_train, y_train))
pipeline

0.9686927560366362


Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('tf_idf', Pipeline(memory=None,
     steps=[('counts', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.9...bf',
  max_iter=-1, probability=False, random_state=0, shrinking=True,
  tol=0.001, verbose=False))])

In [65]:
y_pred_class = pipeline.predict(X_test)
print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
print(metrics.classification_report(y_test, y_pred_class))
print('confusion_matrix: ')
print( metrics.confusion_matrix(y_test, y_pred_class))

accuracy_score:  0.7230359520639148
             precision    recall  f1-score   support

          1       0.87      0.81      0.83       279
          2       0.79      0.29      0.43        65
          3       0.89      0.25      0.40        63
          4       0.38      0.70      0.49        89
          5       0.89      0.55      0.68       128
          6       0.35      0.37      0.36        59
          7       0.73      0.68      0.71       143
          8       0.75      0.85      0.80       676

avg / total       0.75      0.72      0.72      1502

confusion_matrix: 
[[225   2   1   9   5   1   7  29]
 [  7  19   0   8   0   5   2  24]
 [  4   1  16   8   0   2   5  27]
 [  2   0   0  62   1   2   0  22]
 [  6   1   0  12  71   5   5  28]
 [  0   0   0   3   1  22   5  28]
 [  4   0   1   6   0   6  97  29]
 [ 12   1   0  56   2  20  11 574]]


## Perceptron

In [27]:
from sklearn.linear_model import Perceptron

In [28]:
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('tf_idf', Pipeline([
            ('counts', CountVectorizer(max_df=0.95, min_df=2)),
            ('tf_idf', TfidfTransformer()),
            ('chi', SelectKBest(chi2, k=20000))
        ])),
        ('tf', Statskeywords(topk = 1000)),
        ('len_stats', StatsFeatures_cor())
    ])),
    ('standard', StandardScaler(with_mean=False)),
    ('classifier', Perceptron())
])

pipeline.fit(X_train, y_train)
print(pipeline.score(X_train, y_train))
pipeline

0.9928393005828476


Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('tf_idf', Pipeline(memory=None,
     steps=[('counts', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.9...n_jobs=1, penalty=None, random_state=0,
      shuffle=True, tol=None, verbose=0, warm_start=False))])

In [29]:
y_pred_class = pipeline.predict(X_test)
print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
print(metrics.classification_report(y_test, y_pred_class))
print('confusion_matrix: ')
print( metrics.confusion_matrix(y_test, y_pred_class))

accuracy_score:  0.7816245006657789
             precision    recall  f1-score   support

          1       0.85      0.81      0.83       279
          2       0.46      0.55      0.50        65
          3       0.45      0.49      0.47        63
          4       0.67      0.75      0.71        89
          5       0.69      0.73      0.71       128
          6       0.57      0.73      0.64        59
          7       0.79      0.78      0.79       143
          8       0.89      0.84      0.86       676

avg / total       0.79      0.78      0.79      1502

confusion_matrix: 
[[226   6   9   2  12   0   5  19]
 [  7  36   3   4   4   3   0   8]
 [  5   7  31   1   1   5   4   9]
 [  4   5   0  67   1   0   0  12]
 [  8   5   2   3  93   3   5   9]
 [  2   1   0   0   4  43   2   7]
 [  2   5   2   1   4   9 112   8]
 [ 13  13  22  22  15  12  13 566]]


## LogisticRegression

In [30]:
from sklearn.linear_model import LogisticRegression

In [31]:
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('tf_idf', Pipeline([
            ('counts', CountVectorizer(max_df=0.95, min_df=2)),
            ('tf_idf', TfidfTransformer()),
            ('chi', SelectKBest(chi2, k=20000))
        ])),
        ('tf', Statskeywords(topk = 1000)),
        ('len_stats', StatsFeatures_cor())
    ])),
    ('standard', StandardScaler(with_mean=False)),
    ('classifier', LogisticRegression(C=1000.0, random_state=0))
])

pipeline.fit(X_train, y_train)
print(pipeline.score(X_train, y_train))
pipeline

0.9995004163197335


Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('tf_idf', Pipeline(memory=None,
     steps=[('counts', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.9...nalty='l2', random_state=0,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False))])

In [32]:
y_pred_class = pipeline.predict(X_test)
print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
print(metrics.classification_report(y_test, y_pred_class))
print('confusion_matrix: ')
print( metrics.confusion_matrix(y_test, y_pred_class))

accuracy_score:  0.7942743009320905
             precision    recall  f1-score   support

          1       0.76      0.85      0.80       279
          2       0.55      0.32      0.41        65
          3       0.52      0.41      0.46        63
          4       0.85      0.67      0.75        89
          5       0.73      0.71      0.72       128
          6       0.86      0.64      0.74        59
          7       0.88      0.78      0.83       143
          8       0.83      0.90      0.86       676

avg / total       0.79      0.79      0.79      1502

confusion_matrix: 
[[236   2   2   0   9   0   2  28]
 [ 13  21   3   1   7   1   2  17]
 [  5   4  26   0   3   1   1  23]
 [  8   0   0  60   2   0   0  19]
 [ 13   3   1   3  91   0   4  13]
 [  2   0   2   0   3  38   1  13]
 [ 10   2   1   0   1   2 112  15]
 [ 23   6  15   7   8   2   6 609]]


## KNeighborsClassifier

In [38]:
from sklearn.neighbors import KNeighborsClassifier

In [39]:
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('tf_idf', Pipeline([
            ('counts', CountVectorizer(max_df=0.95, min_df=2)),
            ('tf_idf', TfidfTransformer()),
            ('chi', SelectKBest(chi2, k=20000))
        ])),
        ('tf', Statskeywords(topk = 1000)),
        ('len_stats', StatsFeatures_cor())
    ])),
    ('standard', StandardScaler(with_mean=False)),
    ('classifier', KNeighborsClassifier())
])

pipeline.fit(X_train, y_train)
print(pipeline.score(X_train, y_train))
pipeline

0.38018318068276435


Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('tf_idf', Pipeline(memory=None,
     steps=[('counts', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.9...owski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'))])

In [40]:
y_pred_class = pipeline.predict(X_test)
print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
print(metrics.classification_report(y_test, y_pred_class))
print('confusion_matrix: ')
print( metrics.confusion_matrix(y_test, y_pred_class))

accuracy_score:  0.3175765645805593
             precision    recall  f1-score   support

          1       0.21      0.99      0.35       279
          2       1.00      0.02      0.03        65
          3       0.86      0.10      0.17        63
          4       1.00      0.15      0.25        89
          5       0.83      0.27      0.40       128
          6       0.00      0.00      0.00        59
          7       0.92      0.24      0.38       143
          8       0.97      0.17      0.29       676

avg / total       0.77      0.32      0.29      1502

confusion_matrix: 
[[275   0   1   0   0   0   2   1]
 [ 63   1   0   0   0   0   0   1]
 [ 55   0   6   0   1   0   0   1]
 [ 75   0   0  13   1   0   0   0]
 [ 92   0   0   0  34   0   1   1]
 [ 58   0   0   0   1   0   0   0]
 [106   0   0   0   3   0  34   0]
 [561   0   0   0   1   0   0 114]]


## GaussianNB
- 高斯朴素贝叶斯（Gaussian Naive Bayes）
- 多项式朴素贝叶斯（Multinomial Naive Bayes）
- 伯努利朴素贝叶斯（ Bernoulli Naive Bayes）

In [33]:
from sklearn.naive_bayes import GaussianNB

In [35]:
class DenseTransformer(TransformerMixin):

    def transform(self, X, y=None, **fit_params):
        return X.todense()

    def fit_transform(self, X, y=None, **fit_params):
        self.fit(X, y, **fit_params)
        return self.transform(X)

    def fit(self, X, y=None, **fit_params):
        return self

In [36]:
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('tf_idf', Pipeline([
            ('counts', CountVectorizer(max_df=0.95, min_df=2)),
            ('tf_idf', TfidfTransformer()),
            ('chi', SelectKBest(chi2, k=20000))
        ])),
        ('tf', Statskeywords(topk = 1000)),
        ('len_stats', StatsFeatures_cor())
    ])),
    ('standard', StandardScaler(with_mean=False)),
    ('to_dense', DenseTransformer()),
    ('classifier', GaussianNB())
])

pipeline.fit(X_train, y_train)
print(pipeline.score(X_train, y_train))
pipeline

0.9703580349708576


Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('tf_idf', Pipeline(memory=None,
     steps=[('counts', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.9...se', <__main__.DenseTransformer object at 0x7f5088873ac8>), ('classifier', GaussianNB(priors=None))])

In [37]:
y_pred_class = pipeline.predict(X_test)
print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
print(metrics.classification_report(y_test, y_pred_class))
print('confusion_matrix: ')
print( metrics.confusion_matrix(y_test, y_pred_class))

accuracy_score:  0.7197070572569907
             precision    recall  f1-score   support

          1       0.81      0.71      0.75       279
          2       0.41      0.35      0.38        65
          3       0.46      0.41      0.44        63
          4       0.75      0.67      0.71        89
          5       0.71      0.62      0.66       128
          6       0.71      0.37      0.49        59
          7       0.80      0.49      0.61       143
          8       0.72      0.89      0.80       676

avg / total       0.72      0.72      0.71      1502

confusion_matrix: 
[[197   7   9   6  10   0   2  48]
 [  7  23   3   0   1   1   2  28]
 [  2   4  26   0   1   1   2  27]
 [  2   2   0  60   1   0   0  24]
 [ 10   5   2   2  79   1   4  25]
 [  0   0   1   0   2  22   3  31]
 [  3   4   4   1  10   1  70  50]
 [ 23  11  11  11   7   5   4 604]]


In [75]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('tf_idf', Pipeline([
            ('counts', CountVectorizer(max_df=0.95, min_df=2)),
            ('tf_idf', TfidfTransformer()),
            ('chi', SelectKBest(chi2, k=20000))
        ])),
        ('tf', Statskeywords(topk = 1000)),
        ('len_stats', StatsFeatures_cor())
    ])),
    ('standard', StandardScaler(with_mean=False)),
    ('classifier', MultinomialNB())
])

pipeline.fit(X_train, y_train)
print(pipeline.score(X_train, y_train))
pipeline

In [ ]:
y_pred_class = pipeline.predict(X_test)
print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
print(metrics.classification_report(y_test, y_pred_class))
print('confusion_matrix: ')
print( metrics.confusion_matrix(y_test, y_pred_class))

In [44]:
from sklearn.naive_bayes import BernoulliNB

In [45]:
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('tf_idf', Pipeline([
            ('counts', CountVectorizer(max_df=0.95, min_df=2)),
            ('tf_idf', TfidfTransformer()),
            ('chi', SelectKBest(chi2, k=20000))
        ])),
        ('tf', Statskeywords(topk = 1000)),
        ('len_stats', StatsFeatures_cor())
    ])),
    ('standard', StandardScaler(with_mean=False)),
    ('classifier', BernoulliNB())
])

pipeline.fit(X_train, y_train)
print(pipeline.score(X_train, y_train))
pipeline

0.6960865945045795


Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('tf_idf', Pipeline(memory=None,
     steps=[('counts', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.9..._std=True)), ('classifier', BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True))])

In [46]:
y_pred_class = pipeline.predict(X_test)
print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
print(metrics.classification_report(y_test, y_pred_class))
print('confusion_matrix: ')
print( metrics.confusion_matrix(y_test, y_pred_class))

accuracy_score:  0.6531291611185086
             precision    recall  f1-score   support

          1       0.77      0.69      0.73       279
          2       0.47      0.57      0.51        65
          3       0.30      0.43      0.35        63
          4       0.73      0.70      0.71        89
          5       0.68      0.62      0.65       128
          6       0.67      0.73      0.70        59
          7       0.33      0.92      0.49       143
          8       0.97      0.60      0.74       676

avg / total       0.77      0.65      0.68      1502

confusion_matrix: 
[[193   8  13   3  17   1  41   3]
 [  6  37   4   1   2   1  13   1]
 [  2   4  27   0   2   3  25   0]
 [  4   3   0  62   4   1  14   1]
 [  4   8   2   0  80   0  33   1]
 [  0   0   3   0   1  43   9   3]
 [  0   3   2   1   1   2 131   3]
 [ 43  16  39  18  11  13 128 408]]


# 其他模型-组合模型

## Bagging算法
- 使用训练样本的不同子集来建立多个模型（一般是相同类型）
- Bootstrap Aggregation或者叫bagging，从你的训练集多次抽样，在每个样本上进行模型。最后的输出是所有子模型的预测的平均结果。
> - Bagged决策树
    - > Bagging最适合那些具有高度变异的算法。很常用的一个例子就是决策树（不进行剪枝）。
    - > 以下的例子使用 BaggingClassifier和the Classification and Regression Trees 算法 (DecisionTreeClassifier), 创建有100个树的模型。
> - 随机森林    
    - > 随机森林是bagged决策树的一个扩展。从训练样本中可放回的抽样，构建决策树，但是要减少每个分类器之间的相关。就是说，不是贪婪地选取最佳的分割点，而是随机地选择特征子集建立分界点。你可以使用RandomForestClassifier类来构建随机森林模型。 
> - Extra Trees
    - > Extra Trees 也是Bagging方法的一个修正版。使用训练样本直接构建随机树。你可以使用ExtraTreesClassifier类构建Extra Trees。

In [47]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

In [48]:
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('tf_idf', Pipeline([
            ('counts', CountVectorizer(max_df=0.95, min_df=2)),
            ('tf_idf', TfidfTransformer()),
            ('chi', SelectKBest(chi2, k=20000))
        ])),
        ('tf', Statskeywords(topk = 1000)),
        ('len_stats', StatsFeatures_cor())
    ])),
    ('standard', StandardScaler(with_mean=False)),
    ('classifier', BaggingClassifier(base_estimator=DecisionTreeClassifier(), random_state=7))
])

pipeline.fit(X_train, y_train)
print(pipeline.score(X_train, y_train))
pipeline

0.9901748542880933


Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('tf_idf', Pipeline(memory=None,
     steps=[('counts', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.9... n_estimators=10, n_jobs=1, oob_score=False,
         random_state=7, verbose=0, warm_start=False))])

In [49]:
y_pred_class = pipeline.predict(X_test)
print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
print(metrics.classification_report(y_test, y_pred_class))
print('confusion_matrix: ')
print( metrics.confusion_matrix(y_test, y_pred_class))

accuracy_score:  0.8149134487350199
             precision    recall  f1-score   support

          1       0.76      0.87      0.81       279
          2       0.51      0.32      0.40        65
          3       0.63      0.43      0.51        63
          4       0.86      0.73      0.79        89
          5       0.72      0.72      0.72       128
          6       0.78      0.73      0.75        59
          7       0.81      0.83      0.82       143
          8       0.88      0.91      0.90       676

avg / total       0.81      0.81      0.81      1502

confusion_matrix: 
[[243   1   3   2   9   1   0  20]
 [ 16  21   0   3   4   1   4  16]
 [  7   5  27   0   2   3   8  11]
 [  6   1   1  65   2   1   1  12]
 [ 14   2   4   2  92   1   7   6]
 [  1   2   3   0   2  43   1   7]
 [  4   2   3   0   5   3 118   8]
 [ 27   7   2   4  12   2   7 615]]


In [50]:
from sklearn.ensemble import RandomForestClassifier 

In [51]:
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('tf_idf', Pipeline([
            ('counts', CountVectorizer(max_df=0.95, min_df=2)),
            ('tf_idf', TfidfTransformer()),
            ('chi', SelectKBest(chi2, k=20000))
        ])),
        ('tf', Statskeywords(topk = 1000)),
        ('len_stats', StatsFeatures_cor())
    ])),
    ('standard', StandardScaler(with_mean=False)),
    ('classifier', RandomForestClassifier(random_state=7))
])

pipeline.fit(X_train, y_train)
print(pipeline.score(X_train, y_train))
pipeline

0.9946711074104913


Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('tf_idf', Pipeline(memory=None,
     steps=[('counts', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.9...estimators=10, n_jobs=1,
            oob_score=False, random_state=7, verbose=0, warm_start=False))])

In [52]:
y_pred_class = pipeline.predict(X_test)
print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
print(metrics.classification_report(y_test, y_pred_class))
print('confusion_matrix: ')
print( metrics.confusion_matrix(y_test, y_pred_class))

accuracy_score:  0.7976031957390146
             precision    recall  f1-score   support

          1       0.73      0.87      0.79       279
          2       0.59      0.34      0.43        65
          3       0.63      0.35      0.45        63
          4       0.79      0.67      0.73        89
          5       0.83      0.66      0.74       128
          6       0.81      0.64      0.72        59
          7       0.86      0.75      0.80       143
          8       0.83      0.92      0.87       676

avg / total       0.79      0.80      0.79      1502

confusion_matrix: 
[[242   2   2   0   7   0   3  23]
 [ 15  22   1   2   2   1   1  21]
 [  9   1  22   1   2   1   4  23]
 [ 13   1   0  60   0   0   0  15]
 [ 14   7   3   2  85   2   4  11]
 [  6   0   1   0   1  38   2  11]
 [  9   1   1   0   1   4 107  20]
 [ 25   3   5  11   5   1   4 622]]


In [53]:
from sklearn.ensemble import ExtraTreesClassifier

In [54]:
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('tf_idf', Pipeline([
            ('counts', CountVectorizer(max_df=0.95, min_df=2)),
            ('tf_idf', TfidfTransformer()),
            ('chi', SelectKBest(chi2, k=20000))
        ])),
        ('tf', Statskeywords(topk = 1000)),
        ('len_stats', StatsFeatures_cor())
    ])),
    ('standard', StandardScaler(with_mean=False)),
    ('classifier', ExtraTreesClassifier(random_state=7))
])

pipeline.fit(X_train, y_train)
print(pipeline.score(X_train, y_train))
pipeline

0.9995004163197335


Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('tf_idf', Pipeline(memory=None,
     steps=[('counts', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.9..._estimators=10, n_jobs=1,
           oob_score=False, random_state=7, verbose=0, warm_start=False))])

In [55]:
y_pred_class = pipeline.predict(X_test)
print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
print(metrics.classification_report(y_test, y_pred_class))
print('confusion_matrix: ')
print( metrics.confusion_matrix(y_test, y_pred_class))

accuracy_score:  0.8022636484687083
             precision    recall  f1-score   support

          1       0.74      0.88      0.81       279
          2       0.78      0.32      0.46        65
          3       0.76      0.41      0.54        63
          4       0.87      0.65      0.74        89
          5       0.84      0.64      0.73       128
          6       0.83      0.66      0.74        59
          7       0.86      0.75      0.80       143
          8       0.81      0.93      0.86       676

avg / total       0.80      0.80      0.79      1502

confusion_matrix: 
[[245   1   1   1   5   0   0  26]
 [ 17  21   0   1   1   1   2  22]
 [  9   1  26   0   1   3   1  22]
 [  9   0   0  58   1   0   1  20]
 [ 15   1   0   1  82   0   7  22]
 [  2   0   0   0   1  39   3  14]
 [  6   0   2   0   3   3 107  22]
 [ 26   3   5   6   4   1   4 627]]


## Boosting算法
- Boosting集成算法创建一个模型序列，并试图矫正序列中该模型之前模型的错误。一旦模型被创建，模型使用单个模型的准确率作为权重进行整体的预测。
> - AdaBoost
    - > AdaBoost 大概是第一个有效的boosting算法. 一般根据具体实例是否容易被预测给其添加权重，当进行后续模型构建的时候根据权重对该实例进行处理。你可以使用AdaBoostClassifier类构建AdaBoost模型。
> - Stochastic Gradient Boosting
    - > Stochastic Gradient Boosting是最为精细的模型集成技术。它也可能是最有效的提升模型表现的集成技术。你可以使用 GradientBoostingClassifier类构建Gradient Boosting模型。

In [56]:
from sklearn.ensemble import AdaBoostClassifier

In [57]:
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('tf_idf', Pipeline([
            ('counts', CountVectorizer(max_df=0.95, min_df=2)),
            ('tf_idf', TfidfTransformer()),
            ('chi', SelectKBest(chi2, k=20000))
        ])),
        ('tf', Statskeywords(topk = 1000)),
        ('len_stats', StatsFeatures_cor())
    ])),
    ('standard', StandardScaler(with_mean=False)),
    ('classifier', AdaBoostClassifier(random_state=7))
])

pipeline.fit(X_train, y_train)
print(pipeline.score(X_train, y_train))
pipeline

0.7082431307243964


Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('tf_idf', Pipeline(memory=None,
     steps=[('counts', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.9...ithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=7))])

In [58]:
y_pred_class = pipeline.predict(X_test)
print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
print(metrics.classification_report(y_test, y_pred_class))
print('confusion_matrix: ')
print( metrics.confusion_matrix(y_test, y_pred_class))

accuracy_score:  0.6930758988015979
             precision    recall  f1-score   support

          1       0.70      0.71      0.70       279
          2       0.23      0.11      0.15        65
          3       0.32      0.11      0.16        63
          4       0.67      0.64      0.66        89
          5       0.35      0.61      0.45       128
          6       0.73      0.59      0.65        59
          7       0.66      0.55      0.60       143
          8       0.84      0.86      0.85       676

avg / total       0.69      0.69      0.69      1502

confusion_matrix: 
[[197   3   1   8  12   1  12  45]
 [ 13   7   2   5  16   3   5  14]
 [  2   5   7   0  23   4   5  17]
 [  3   0   0  57  16   0   0  13]
 [ 23   2   4   4  78   0   9   8]
 [  1   3   1   0   9  35   4   6]
 [ 12   4   2   1  35   2  78   9]
 [ 32   6   5  10  32   3   6 582]]


In [59]:
from sklearn.ensemble import GradientBoostingClassifier

In [60]:
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('tf_idf', Pipeline([
            ('counts', CountVectorizer(max_df=0.95, min_df=2)),
            ('tf_idf', TfidfTransformer()),
            ('chi', SelectKBest(chi2, k=20000))
        ])),
        ('tf', Statskeywords(topk = 1000)),
        ('len_stats', StatsFeatures_cor())
    ])),
    ('standard', StandardScaler(with_mean=False)),
    ('classifier', GradientBoostingClassifier(random_state=7))
])

pipeline.fit(X_train, y_train)
print(pipeline.score(X_train, y_train))
pipeline

0.9675270607826811


Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('tf_idf', Pipeline(memory=None,
     steps=[('counts', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.9...         presort='auto', random_state=7, subsample=1.0, verbose=0,
              warm_start=False))])

In [61]:
y_pred_class = pipeline.predict(X_test)
print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
print(metrics.classification_report(y_test, y_pred_class))
print('confusion_matrix: ')
print( metrics.confusion_matrix(y_test, y_pred_class))

accuracy_score:  0.8422103861517976
             precision    recall  f1-score   support

          1       0.81      0.90      0.85       279
          2       0.68      0.49      0.57        65
          3       0.72      0.46      0.56        63
          4       0.93      0.71      0.80        89
          5       0.76      0.73      0.75       128
          6       0.84      0.73      0.78        59
          7       0.92      0.83      0.87       143
          8       0.87      0.94      0.90       676

avg / total       0.84      0.84      0.84      1502

confusion_matrix: 
[[250   3   1   0   7   0   1  17]
 [ 12  32   1   2   3   1   1  13]
 [  2   1  29   0   6   4   3  18]
 [  6   3   0  63   3   0   0  14]
 [ 11   3   4   0  94   0   5  11]
 [  0   0   2   0   3  43   0  11]
 [  4   2   1   0   1   2 119  14]
 [ 25   3   2   3   6   1   1 635]]


## Voting算法
- Voting是组合多个算法进行预测的一个最简单的方法。
- 首先使用你的训练集分别建立模型，Voting类用于组合你的模型，平均多个模型的结果来进行预测。
- 子模型的预测可以进行加权，但是手动的给分类器加上权重还是比较困难的。更为高级的方法是让模型自己学习给子模型加权。
- 这个所谓的stacked aggregation方法目前在 scikit-learn还不能实现。可以使用VotingClassifier类建立voting模型。
- 以下例子组合logistic 回归，分类和回归树以及SVM来进行预测。

In [23]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import VotingClassifier

In [18]:
help(VotingClassifier)

Help on class VotingClassifier in module sklearn.ensemble.voting_classifier:

class VotingClassifier(sklearn.utils.metaestimators._BaseComposition, sklearn.base.ClassifierMixin, sklearn.base.TransformerMixin)
 |  Soft Voting/Majority Rule classifier for unfitted estimators.
 |  
 |  .. versionadded:: 0.17
 |  
 |  Read more in the :ref:`User Guide <voting_classifier>`.
 |  
 |  Parameters
 |  ----------
 |  estimators : list of (string, estimator) tuples
 |      Invoking the ``fit`` method on the ``VotingClassifier`` will fit clones
 |      of those original estimators that will be stored in the class attribute
 |      ``self.estimators_``. An estimator can be set to `None` using
 |      ``set_params``.
 |  
 |  voting : str, {'hard', 'soft'} (default='hard')
 |      If 'hard', uses predicted class labels for majority rule voting.
 |      Else if 'soft', predicts the class label based on the argmax of
 |      the sums of the predicted probabilities, which is recommended for
 |      an 

In [24]:
clf1 = GradientBoostingClassifier()
clf2 = BernoulliNB()
clf3 = MultinomialNB()

estimators = [('gdbt', clf1), ('bnb', clf2), ('mnb', clf3)]
ensemble = VotingClassifier(estimators, n_jobs = -1, voting = 'soft')

In [25]:
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('tf_idf', Pipeline([
            ('counts', CountVectorizer(max_df=0.95, min_df=2)),
            ('tf_idf', TfidfTransformer()),
            ('chi', SelectKBest(chi2, k=20000))
        ])),
        ('tf', Statskeywords(topk = 1000)),
        ('len_stats', StatsFeatures_cor())
    ])),
    ('standard', StandardScaler(with_mean=False)),
    ('classifier', ensemble)
])

pipeline.fit(X_train, y_train)
print(pipeline.score(X_train, y_train))
pipeline

0.9358583936048724


Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('tf_idf', Pipeline(memory=None,
     steps=[('counts', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.9...=None, fit_prior=True))],
         flatten_transform=None, n_jobs=-1, voting='soft', weights=None))])

In [26]:
y_pred_class = pipeline.predict(X_test)
print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
print(metrics.classification_report(y_test, y_pred_class))
print('confusion_matrix: ')
print( metrics.confusion_matrix(y_test, y_pred_class))

accuracy_score:  0.8104749223257879
             precision    recall  f1-score   support

          1       0.81      0.86      0.84       434
          2       0.55      0.58      0.56        97
          3       0.55      0.67      0.60        98
          4       0.81      0.87      0.84       137
          5       0.77      0.74      0.76       177
          6       0.75      0.76      0.76        88
          7       0.68      0.90      0.78       210
          8       0.93      0.81      0.87      1012

avg / total       0.82      0.81      0.81      2253

confusion_matrix: 
[[375  11   5   3   9   1  11  19]
 [ 16  56   3   2   4   3   6   7]
 [  4   5  66   1   3   3  10   6]
 [  5   3   0 119   4   0   1   5]
 [  7   9   4   1 131   0  15  10]
 [  0   0   3   0   4  67   5   9]
 [  3   3   2   0   3   3 189   7]
 [ 52  15  38  21  12  12  39 823]]


# 模型保存

In [34]:
from sklearn.externals import joblib
joblib.dump(pipeline, "model/circ_8classifier_0730_1.pkl.z")

['model/circ_8classifier_0730_1.pkl.z']

# 保存本文件

In [56]:
if 0:
    import datetime as dt
    
    def output_HTML(read_file, output_file):
        from nbconvert import HTMLExporter
        import codecs
        import nbformat
        exporter = HTMLExporter()
        # read_file is '.ipynb', output_file is '.html'
        output_notebook = nbformat.read(read_file, as_version=4)
        output, resources = exporter.from_notebook_node(output_notebook)
        codecs.open(output_file, 'w', encoding='utf-8').write(output)

    html_file_folder = 'html_files'
    if not os.path.exists(html_file_folder):
        os.makedirs(html_file_folder)

    today = dt.datetime.now().strftime('%Y%m%d')
    current_file = 'circ_cor_model_2_train.ipynb'
    output_file = 'html_files\%s_%s.html'%(os.path.splitext(current_file)[0], today)
    output_HTML(current_file, output_file)